In [0]:
import sys

sys.path.append("../include_utils/")

#from IPython.parallel import Client
import ipyparallel as ipp
import os, time
import include_utils as u
import pandas as pd
import numpy as np
import scipy as sp
import numbers
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import vcf
from sklearn import preprocessing
from subprocess import Popen, PIPE
import seaborn as sns
from IPython.display import FileLink
import urllib.request as urllib2
import pickle
import traceback
from pandas import Series, DataFrame
import gzip
import warnings
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)
%config InlineBackend.figure_format = 'retina'
from Bio import SeqIO
import pysam
from collections import OrderedDict, namedtuple
import operator
import multiprocessing as mp
from ipyparallel import Client
from Bio import SeqIO
import pyfaidx
import pyfasta
import textwrap
import tempfile
%matplotlib inline
from collections import Counter
import pysam

In [0]:
filedir = "/gpfs_fs/home/eckertlab/Mitra/mapping_bwa/trimmed/mapped/split_parallel/collapsed"

In [0]:
cd $filedir

In [0]:
mapped = !ls *mapped.bam

In [0]:
samtools = "/home/cfriedline/bin/samtools"

In [0]:
from subprocess import Popen, PIPE, STDOUT

In [0]:
rc = Client(profile = "sge")
dv = rc[:]
lv = rc.load_balanced_view()
len(dv)

In [0]:
with dv.sync_imports():
    from subprocess import Popen, PIPE, STDOUT
    import os, sys
    import socket

In [0]:
%px cd $filedir

In [0]:
def get_contigs(bam):
    contigs = set()
    cmd = "samtools view %s" % bam
    sys.stderr.write("%s: %s\n" % (socket.gethostname(), cmd))
    p = Popen(cmd, stdout=PIPE, shell=True)
    for line in p.stdout:
        d = line.decode().split("\t")
        contigs.add(d[2])
    return contigs

In [0]:
dv.push({'samtools': samtools,
        'get_contigs': get_contigs}).r;

In [0]:
jobs = []
for b in mapped:
    jobs.append(lv.apply_async(get_contigs, b))

In [0]:
len(jobs)

In [0]:
np.sum([x.ready() for x in jobs])

In [0]:
all_contigs = set()
for j in jobs:
    for contig in j.r:
        all_contigs.add(contig)

In [0]:
contig_counts = Counter()
for j in jobs:
    for contig in j.r:
        contig_counts[contig] += 1

In [0]:
pwd

In [0]:
all_contigs_sav = "all_contigs.pkl"

In [0]:
contig_counts_sav = "contig_counts.pkl"

In [0]:
pickle.dump(all_contigs, open(all_contigs_sav, "wb"), pickle.HIGHEST_PROTOCOL)

In [0]:
pickle.dump(contig_counts, open(contig_counts_sav, "wb"), pickle.HIGHEST_PROTOCOL)

In [0]:
all_contigs = pickle.load(open(all_contigs_sav, "rb"))

In [0]:
contig_counts = pickle.load(open(contig_counts_sav, "rb"))

In [0]:
assembly = "/home/cfriedline/eckertlab/SugarPine_genome/pila.v1.0.scafSeq"

In [0]:
#count = !grep -c ">" $assembly

In [0]:
count = 58407655 #save for later

In [0]:
len(all_contigs)

In [0]:
len(all_contigs), len(all_contigs)/count

In [0]:
mapped_fasta = os.path.join(filedir, "%s_mapped.fasta" % os.path.basename(assembly))

In [0]:
mapped_fasta

In [0]:
faSomeRecords = "/home/cfriedline/g/src/kentUtils/bin/faSomeRecords"

In [0]:
with open("all_contigs.txt", "w") as o:
    for c in all_contigs:
        o.write("{}\n".format(c))

In [0]:
## Run this in a terminal
"{} {} {} {}".format(faSomeRecords, assembly, "all_contigs.txt", mapped_fasta)

In [0]:
with open("contigs.bed", "w") as o:
    for rec in SeqIO.parse(mapped_fasta, "fasta"):
        o.write("%s\t%d\t%d\n" % (rec.name, 0, len(rec)))